In [1]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
!pip install datasets

In [7]:
import tempfile
import logging
import random
import os
import yaml
import time
import torch
import transformers
import pandas as pd

In [8]:
from transformers import AutoTokenizer,AutoModelForCausalLM,TrainingArguments,Trainer
import pandas as pd
import datasets

In [9]:
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-70m')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
data1 = '/content/data.json'
df = pd.read_json(data1)
df1 = df.to_dict()
df1

{'question': {0: ' what is the company law about migrants',
  1: 'what is the main idea behind being tough about migrants'},
 'answer': {0: ' UF company has straight and tough rules about the migrants, like they can not pay with cash or can not get the normal customers offers',
  1: 'it is because the lake of saftey in this side'}}

In [12]:
for i in range(len(df1['question'])):
  if 'question' in df1 and 'answer' in df1:
    example = df1['question'][i] + df1['answer'][i]

In [13]:
prompt_template = """
###answer the following question with the user data
{question}
###Answer:
"""

In [14]:
data_q_a =[]
for i in range(len(df1['question'])):
  question = df1['question'][i]
  answer = df1['answer'][i]
  text = prompt_template.format(question = question)
  data_q_a.append({'question':text,'answer':answer})

In [15]:
data_q_a

[{'question': '\n###answer the following question with the user data\n what is the company law about migrants\n###Answer:\n',
  'answer': ' UF company has straight and tough rules about the migrants, like they can not pay with cash or can not get the normal customers offers'},
 {'question': '\n###answer the following question with the user data\nwhat is the main idea behind being tough about migrants\n###Answer:\n',
  'answer': 'it is because the lake of saftey in this side'}]

In [16]:
def tokenizing_function(examples):

    if "question" in examples and "answer" in examples:
      text = examples["question"][0] + examples["answer"][0]

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs

In [17]:
finetuning_dataset= datasets.load_dataset("json", data_files=data1, split="train")

tokenized_dataset = finetuning_dataset.map(
    tokenizing_function,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [18]:
print(tokenized_dataset)

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 2
})


In [19]:
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

In [20]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})

In [21]:
base_model = AutoModelForCausalLM.from_pretrained('EleutherAI/pythia-70m')

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

In [22]:
import torch
device_count = torch.cuda.device_count()
if device_count > 0:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [23]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [24]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )


  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [25]:
max_steps = 3
trained_model_name = f"docs_{max_steps}_steps"
output_dir = trained_model_name

In [26]:
import transformers
training_args = TrainingArguments(

  learning_rate=1.0e-5,
  num_train_epochs=1,
  max_steps=max_steps,
  per_device_train_batch_size=1,
  output_dir=output_dir,
)

In [ ]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset = split_dataset['test']

)

In [29]:
training_output = trainer.train()

Step,Training Loss


In [31]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: docs_3_steps/final


In [33]:
finetuned_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
finetuned_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [39]:
test_question = 'what is your job?'

'what is your job?'

In [41]:
print(inference(test_question, finetuned_slightly_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m not sure if I’m
